# Data Processing

In [1]:
import geopy
from geopy.geocoders import Nominatim
import time
from pprint import pprint

import pandas as pd
import re

from ipyleaflet import Map, AntPath, MeasureControl
import ipywidgets
from vega_datasets import data as vds

app = Nominatim(timeout=10, user_agent = 'bebs')

In [2]:
#example

home_address = 'Urussui 70, Itaim Bibi, Sao Paulo, Brazil'
home = app.geocode(home_address)
print(home)

example_address = 'melbourne australia'
example = app.geocode(example_address)
print(example)

Rua Urussuí, Vila Olímpia, Itaim Bibi, São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 04542-050, Brasil
Melbourne, City of Melbourne, Victoria, Australia


In [3]:
jej = pd.read_excel(r"C:\Users\srjbe\Downloads\JEJSupply.xlsx", index_col=0)
jej = jej.reset_index()
#jej

In [4]:
jej['place'] =jej['BusinessDescription'].str.split('headquartered in')
jej['place'] = jej['place'].str[1]
#jej['place']

In [5]:
jej['address'] = jej['place'].apply(app.geocode)
#jej['address']

In [6]:
jej['point'] = jej['address'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
jej[['lat', 'long', 'alt']] = pd.DataFrame(jej['point'].tolist(), index=jej.index)

In [7]:
jej = jej.drop(['place', 'address', 'point'], axis=1)

# Mapping

In [12]:
import folium
import geopandas as gpd
import numpy as np
from folium.features import GeoJsonTooltip

In [14]:
location = jej[['lat', 'long']]
location = location.dropna()
location

,lat,long
0,35.021041,135.755607
1,35.021041,135.755607
2,40.014986,-105.270545
3,38.626426,-90.199585
4,38.626426,-90.199585
...,...,...
105,22.991475,120.212423
106,55.605293,13.000157
107,-23.550651,-46.633382
108,42.491730,-71.281795


In [19]:
this_map = folium.Map(prefer_canvas=True)

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.lat, point.long],
                        radius=2,
                        weight=5).add_to(this_map)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
location.apply(plotDot, axis = 1)

#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

this_map